# Exercise Sheet 9
Decrypt an RSA Message using the Period Finding Algorithm
## Task 9.1: RSA Encryption
Given is an RSA system with the parameters p = 11,q = 5 and the public key pk = 17.
- a) Encrypt the message m = “&” with the public key to obtain the ciphertext c. To do this, use the ASCII code of
m = “&” = 38.



In [2]:
from sympy import mod_inverse
import random
import math

In [50]:
m = 38

# Choose two random prime numbers p and q.
p = 11
q = 5
# Calculate the product (RSA modulus) N = pq and Euler’s totient function φ(N) = (p − 1)(q − 1).
n = p * q  # n = 55
phi_n = (p - 1)*(q - 1)

# Choose a number pk that is not related to φ(N) such that pk < φ(N).
pk = 17  # = random.randint(1, phi_n-1)

print(f"{math.gcd(17, 40)=}; pk and phi_N needs to be coprime!")
# Calculate the multiplicative inverse sk of pk with respect to φ(N), i.e., solve the equation
# pk · sk ≡ 1 mod φ(N).
sk = mod_inverse(pk, phi_n)

# (pk, N) is published as the public key, sk is the secret key. The remaining variables are no longer required.
pk = (pk, n)


# Encryption
c = pow(m, pk[0], n)

c # The cifer

math.gcd(17, 40)=1; pk and phi_N needs to be coprime!


3

- b) Show that the secret key sk = 33 is correct by decrypting c with it. You should get m = 38 again.

In [52]:
# Decryption
decrypted_m = pow(c, sk, n)

print(f"{c = }, {decrypted_m = }")

c = 3, decrypted_m = 38


## Task 9.2: RSA Encryption
Decoding using the Period Finding Algorithm
In this task, you will decrypt the ciphertext c from the previous task without knowing the secret key sk. Instead, you will calculate a “second key” for this ciphertext.
- a) Calculate the period r for cx ≡1 mod N. For example, you can create a table of values in WolframAlpha11 with the following command: Table[{x,Mod[c^x, N]}, {x, 1, 100, 1}]. The smallest value that maps to 1 is the
period.

From the previous task we know that $c = 48$, and $N = p \times q = 55$


In [45]:
[(x, (48 ** x) % 55) for x in range(25)] # Python equivalent for Table[{x,Mod[c^x, N]}, {x, 1, 25, 1}]

[(0, 1),
 (1, 48),
 (2, 49),
 (3, 42),
 (4, 36),
 (5, 23),
 (6, 4),
 (7, 27),
 (8, 31),
 (9, 3),
 (10, 34),
 (11, 37),
 (12, 16),
 (13, 53),
 (14, 14),
 (15, 12),
 (16, 26),
 (17, 38),
 (18, 9),
 (19, 47),
 (20, 1),
 (21, 48),
 (22, 49),
 (23, 42),
 (24, 36)]

In [14]:
r = 20

- b) Find a “second key” ˆsk for which pk · ˆsk ≡1 mod r.

$pk=17$

$r=20$

$$17 \times \hat{sk} ≡ 1 (\mod20)$$


Thus, let us first find out if 20 and 17 are co-prime (the greatest common divisor is 1):

$$
\begin{aligned}
20&=1\times17+3\\
17&=5\times3+2\\
3&=1\times2+1\\
2&=2\times1+0\\
\end{aligned}
$$

As they are co-prime, we now use the Extended Euclidean Algorithm to express 1 as a combination of 17 and 20:

$$
\begin{aligned}
1&=3-1\times2\\
1&=3-1\times(17-5\times3)\\
1&=3-17+5\times3\\
1&=6\times3-1\times17\\
1&=6\times20-6\times17-1\times17\\
1&=6\times20-7\times17\\
\end{aligned}
$$

we need $\hat{sk}$ s.t. $-7 \mod 20 = \hat{sk} \mod 20$, with $\hat{sk} > 0$. Thus $-7+20=13=\hat{sk}$ is a good solution.

In [53]:
# Same in Python :)
second_sk = mod_inverse(pk[0], r)

In [54]:
second_sk

13

- c) Show that the calculated secret key ˆsk works by decrypting c with the secret key. You should get m = 38 again.


In [56]:
print(pk[0], c)

# Decryption with second key
decrypted_m = pow(c, second_sk, n)

print(f"{c = }, {decrypted_m = }")

17 3
c = 3, decrypted_m = 38


## Task 9.2: Prime Factorization using the Period Finding Algorithm
### Compile the following cell first to create all necessary classes

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from abc import ABC

import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

def run(circuit):
    # returns how often a measurement result was observed
    sim = AerSimulator()
    job = sim.run(circuit.decompose(reps=6))
    r = job.result().get_counts()
    return r

# This code is part of Qiskit.
#
# (C) Copyright IBM 2019, 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""Shor's factoring algorithm."""

from typing import Optional, Union, Tuple
import math
import array
import fractions
import logging as logger
import numpy as np

from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.circuit import Gate, Instruction, ParameterVector
from qiskit.circuit.library import QFT


class MyShor(ABC):
    """Shor's factoring algorithm.

    Shor's Factoring algorithm is one of the most well-known quantum algorithms and finds the
    prime factors for input integer :math:`N` in polynomial time.

    The input integer :math:`N` to be factored is expected to be odd and greater than 2.
    Even though this implementation is general, its capability will be limited by the
    capacity of the simulator/hardware. Another input integer :math:`a`  can also be supplied,
    which needs to be a co-prime smaller than :math:`N` .
    """

    def __init__(self,
                 N: int = 15,
                 a: int = 2) -> None:
        """
        Args:
            N: The integer to be factored, has a min. value of 3.
            a: Any integer that satisfies 1 < a < N and gcd(a, N) = 1.
            quantum_instance: Quantum Instance or Backend

         Raises:
            ValueError: Invalid input
        """
        self._validate_min('N', N, 3)
        self._validate_min('a', a, 2)
        self._n = None  # type: Optional[int]
        self._up_qreg = None
        self._down_qreg = None  # type: Optional[QuantumRegister]
        self._aux_qreg = None  # type: Optional[QuantumRegister]

        # check the input integer
        if N < 1 or N % 2 == 0:
            raise ValueError('The input needs to be an odd integer greater than 1.')

        self._N = N

        if a >= N or math.gcd(a, self._N) != 1:
            raise ValueError('The integer a needs to satisfy a < N and gcd(a, N) = 1.')

        self._a = a

        self._ret = {"factors": [], "total_counts": 0, "successful_counts": 0}

        # check if the input integer is a power
        tf, b, p = self._is_power(N, return_decomposition=True)
        if tf:
            logger.info('The input integer is a power: %s=%s^%s.', N, b, p)
            self._ret['factors'].append(b)

        self._qft = QFT(do_swaps=False).to_instruction()
        self._iqft = self._qft.inverse()

        self._phi_add_N = None  # type: Optional[Gate]
        self._iphi_add_N = None

    def _is_power(self, num, return_decomposition=False):
        """
        Check if num is a perfect power in O(n^3) time, n=ceil(logN)
        """
        b = 2
        while (2**b) <= num:
            a = 1
            c = num
            while (c - a) >= 2:
                m = int((a + c) / 2)
                if (m**b) < (num + 1):
                    p = int(m**b)
                else:
                    p = int(num + 1)
                if int(p) == int(num):
                    if return_decomposition:
                        return True, int(m), int(b)
                    else:
                        return True
                if p < num:
                    a = int(m)
                else:
                    c = int(m)
            b = b + 1
        if return_decomposition:
            return False, num, 1
        else:
            return False

    def _summarize_circuits(self, circuits):
        """Summarize circuits based on QuantumCircuit, and five metrics are summarized.
            - Number of qubits
            - Number of classical bits
            - Number of operations
            - Depth of circuits
            - Counts of different gate operations
        The average statistic of the first four is provided if multiple circuits are provided.
        Args:
            circuits (QuantumCircuit or [QuantumCircuit]): the to-be-summarized circuits
        Returns:
            str: a formatted string records the summary
        """
        if not isinstance(circuits, list):
            circuits = [circuits]
        ret = ""
        ret += f"Submitting {len(circuits)} circuits.\n"
        ret += "============================================================================\n"
        stats = np.zeros(4)
        for i, circuit in enumerate(circuits):
            depth = circuit.depth()
            size = circuit.size()
            num_qubits = sum(reg.size for reg in circuit.qregs)
            num_clbits = sum(reg.size for reg in circuit.cregs)
            op_counts = circuit.count_ops()
            stats[0] += num_qubits
            stats[1] += num_clbits
            stats[2] += size
            stats[3] += depth
            ret = "".join(
                [
                    ret,
                    "{}-th circuit: {} qubits, {} classical bits and {} "
                    "operations with depth {}\nop_counts: {}\n".format(
                        i, num_qubits, num_clbits, size, depth, op_counts
                    ),
                ]
            )
        if len(circuits) > 1:
            stats /= len(circuits)
            ret = "".join(
                [
                    ret,
                    "Average: {:.2f} qubits, {:.2f} classical bits and {:.2f} "
                    "operations with depth {:.2f}\n".format(stats[0], stats[1], stats[2], stats[3]),
                ]
            )
        ret += "============================================================================\n"
        return ret

    def _validate_min(self, name: str, value: float, minimum: float) -> None:
        """
        Args:
            name: value name.
            value: value to check.
            minimum: minimum value allowed.
        Raises:
            ValueError: invalid value
        """
        if value < minimum:
            raise ValueError(f"{name} must have value >= {minimum}, was {value}")

    def _get_angles(self, a: int) -> np.ndarray:
        """Calculates the array of angles to be used in the addition in Fourier Space."""
        s = bin(int(a))[2:].zfill(self._n + 1)
        angles = np.zeros([self._n + 1])
        for i in range(0, self._n + 1):
            for j in range(i, self._n + 1):
                if s[j] == '1':
                    angles[self._n - i] += math.pow(2, -(j - i))
            angles[self._n - i] *= np.pi
        return angles[::-1]

    @staticmethod
    def _phi_add_gate(size: int, angles: Union[np.ndarray, ParameterVector]) -> Gate:
        """Gate that performs addition by a in Fourier Space."""
        circuit = QuantumCircuit(size, name="phi_add")
        for i, angle in enumerate(angles):
            circuit.p(angle, i)
        return circuit.to_gate()

    def _double_controlled_phi_add_mod_N(self,
                                         num_qubits: int,
                                         angles: Union[np.ndarray, ParameterVector]
                                         ) -> QuantumCircuit:
        """Creates a circuit which implements double-controlled modular addition by a."""
        circuit = QuantumCircuit(num_qubits, name="phi_add")

        ctl_up = 0
        ctl_down = 1
        ctl_aux = 2

        # get qubits from aux register, omitting the control qubit
        qubits = range(3, num_qubits)
        
        qft = QFT(len(qubits)).to_instruction()
        iqft = qft.inverse()

        # store the gates representing addition/subtraction by a in Fourier Space
        phi_add_a = self._phi_add_gate(len(qubits), angles)
        iphi_add_a = phi_add_a.inverse()

        circuit.append(phi_add_a.control(2), [ctl_up, ctl_down, *qubits])
        circuit.append(self._iphi_add_N, qubits)
        circuit.append(iqft, qubits)

        circuit.cx(qubits[0], ctl_aux)

        circuit.append(qft, qubits)
        circuit.append(self._phi_add_N, qubits)
        circuit.append(iphi_add_a.control(2), [ctl_up, ctl_down, *qubits])
        circuit.append(iqft, qubits)

        circuit.x(qubits[0])
        circuit.cx(qubits[0], ctl_aux)
        circuit.x(qubits[0])

        circuit.append(qft, qubits)
        circuit.append(phi_add_a.control(2), [ctl_up, ctl_down, *qubits])
        
        return circuit

    def _controlled_multiple_mod_N(self, num_qubits: int, a: int) -> Instruction:
        """Implements modular multiplication by a as an instruction."""
        circuit = QuantumCircuit(
            num_qubits, name="multiply_by_{}_mod_{}".format(a % self._N, self._N)
        )
        down = circuit.qubits[1: self._n + 1]
        aux = circuit.qubits[self._n + 1:]
        qubits = [aux[i] for i in reversed(range(self._n + 1))]
        ctl_up = 0
        ctl_aux = aux[-1]

        angle_params = ParameterVector("angles", length=len(aux) - 1)
        double_controlled_phi_add = self._double_controlled_phi_add_mod_N(
            len(aux) + 2, angle_params
        )
        idouble_controlled_phi_add = double_controlled_phi_add.inverse()
        
        qft = QFT(len(qubits)).to_instruction()
        iqft = qft.inverse()
        
        circuit.append(qft, qubits)

        # perform controlled addition by a on the aux register in Fourier space
        for i, ctl_down in enumerate(down):
            a_exp = (2 ** i) * a % self._N
            angles = self._get_angles(a_exp)
            bound = double_controlled_phi_add.assign_parameters({angle_params: angles})
            circuit.append(bound, [ctl_up, ctl_down, ctl_aux, *qubits])

        circuit.append(iqft, qubits)

        # perform controlled subtraction by a in Fourier space on both the aux and down register
        for j in range(self._n):
            circuit.cswap(ctl_up, down[j], aux[j])
        circuit.append(qft, qubits)

        a_inv = self.modinv(a, self._N)
        for i in reversed(range(len(down))):
            a_exp = (2 ** i) * a_inv % self._N
            angles = self._get_angles(a_exp)
            bound = idouble_controlled_phi_add.assign_parameters({angle_params: angles})
            circuit.append(bound, [ctl_up, down[i], ctl_aux, *qubits])

        circuit.append(iqft, qubits)
        return circuit.to_instruction()

    def construct_circuit(self, measurement: bool = True) -> QuantumCircuit:
        """Construct circuit.

        Args:
            measurement: Boolean flag to indicate if measurement should be included in the circuit.

        Returns:
            Quantum circuit.
        """

        # Get n value used in Shor's algorithm, to know how many qubits are used
        self._n = math.ceil(math.log(self._N, 2))
        self._qft.num_qubits = self._n + 1
        self._iqft.num_qubits = self._n + 1

        # quantum register where the sequential QFT is performed
        self._up_qreg = QuantumRegister(2 * self._n, name='up')
        # quantum register where the multiplications are made
        self._down_qreg = QuantumRegister(self._n, name='down')
        # auxiliary quantum register used in addition and multiplication
        self._aux_qreg = QuantumRegister(self._n + 2, name='aux')

        # Create Quantum Circuit
        circuit = QuantumCircuit(self._up_qreg,
                                 self._down_qreg,
                                 self._aux_qreg,
                                 name="Shor(N={}, a={})".format(self._N, self._a))

        # Create gates to perform addition/subtraction by N in Fourier Space
        self._phi_add_N = self._phi_add_gate(self._aux_qreg.size - 1, self._get_angles(self._N))
        self._iphi_add_N = self._phi_add_N.inverse()

        # Create maximal superposition in top register
        circuit.h(self._up_qreg)

        # Initialize down register to 1
        circuit.x(self._down_qreg[0])

        # Apply the multiplication gates as showed in
        # the report in order to create the exponentiation
        for i, ctl_up in enumerate(self._up_qreg):  # type: ignore
            a = int(pow(self._a, pow(2, i)))
            controlled_multiple_mod_N = self._controlled_multiple_mod_N(
                len(self._down_qreg) + len(self._aux_qreg) + 1, a,
            )
            circuit.append(
                controlled_multiple_mod_N, [ctl_up, *self._down_qreg, *self._aux_qreg]
            )
        
        # Apply inverse QFT
        iqft = QFT(len(self._up_qreg)).inverse().reverse_bits().to_instruction()
        circuit.append(iqft, self._up_qreg)

        if measurement:
            up_cqreg = ClassicalRegister(2 * self._n, name='m')
            circuit.add_register(up_cqreg)
            circuit.measure(self._up_qreg, up_cqreg)

        logger.info(self._summarize_circuits(circuit))

        return circuit

    @staticmethod
    def modinv(a: int, m: int) -> int:
        """Returns the modular multiplicative inverse of a with respect to the modulus m."""
        def egcd(a: int, b: int) -> Tuple[int, int, int]:
            if a == 0:
                return b, 0, 1
            else:
                g, y, x = egcd(b % a, a)
                return g, x - (b // a) * y, y

        g, x, _ = egcd(a, m)
        if g != 1:
            raise ValueError("The greatest common divisor of {} and {} is {}, so the "
                             "modular inverse does not exist.".format(a, m, g))
        return x % m

    def get_period(self, keylist: list):
        period = []
        for measurement in keylist:
            """Apply the continued fractions to find r and the gcd to find the desired factors."""
            x_final = int(measurement, 2)
            logger.info('In decimal, x_final value for this result is: %s.', x_final)

            if x_final <= 0:
                fail_reason = 'x_final value is <= 0, there are no continued fractions.'
            else:
                fail_reason = None
                logger.debug('Running continued fractions for this case.')

            # Calculate T and x/T
            T_upper = len(measurement)
            T = pow(2, T_upper)
            x_over_T = x_final / T

            # Cycle in which each iteration corresponds to putting one more term in the
            # calculation of the Continued Fraction (CF) of x/T

            # Initialize the first values according to CF rule
            i = 0
            b = array.array('i')
            t = array.array('f')

            b.append(math.floor(x_over_T))
            t.append(x_over_T - b[i])

            exponential = 0.0
                
            
            while i < self._N and fail_reason is None:
                # From the 2nd iteration onwards, calculate the new terms of the CF based
                # on the previous terms as the rule suggests
                if i > 0:
                    b.append(math.floor(1 / t[i - 1]))
                    t.append((1 / t[i - 1]) - b[i])  # type: ignore

                # Calculate the denominator of the CF using the known terms
                denominator = self._calculate_continued_fraction(b)

                # Increment i for next iteration
                i += 1

                if denominator < 1000:
                    try:
                        exponential = pow(self._a, denominator / 2)
                    except OverflowError:
                        pass

                # Check if the value is too big or not
                if exponential > 1000000000:
                    fail_reason = 'denominator of continued fraction is too big.'
                if pow(self._a, denominator, self._N) == 1: 
                    period.append(denominator)
                if t[len(t)-1] == 0:
                    break
        return min(period)
        

    @staticmethod
    def _calculate_continued_fraction(b: array.array) -> int:
        """Calculate the continued fraction of x/T from the current terms of expansion b."""

        x_over_T = 0

        for i in reversed(range(len(b) - 1)):
            x_over_T = 1 / (b[i + 1] + x_over_T)

        x_over_T += b[0]

        # Get the denominator from the value obtained
        frac = fractions.Fraction(x_over_T).limit_denominator()

        logger.debug('Approximation number %s of continued fractions:', len(b))
        logger.debug("Numerator:%s \t\t Denominator: %s.", frac.numerator, frac.denominator)
        return frac.denominator
    
    def _run(self):
        return 0

## Exercise 9.3 c-i)

> *Note:* This computation should take no longer than five minutes.

> *Hint:* 
> First construct the quantum circuit. 
> Then use the function `run(circuit)` to run the quantum circuit. 
> Then compute the period dependent on the measurement result count.
> There are two functions of the class `MyShor` that you need for this.

In [ ]:
a = 2
N = 21
#Todo: Put your code for 10.3 c-i here

## Exercise 9.3 c-ii)
    
> *Note:* The computation of the factors might take some time (around one hour). First test your function with small numbers (e.g., $N=21$) before testing with $N=55$.

> *Hint:* You need to implement the EEA and the classical part of Shor's algorithm.

In [ ]:
def breakRSA(pk: int, N: int):
    #Todo: Put your code for 10.3 c-ii here
    pass

In [ ]:
#Input
breakRSA(pk=3,N=55)